In [1]:
import re
import pandas as pd
import spacy
import stanza
# Stanza 초기화
stanza.download('id')  # 영어 모델 다운로드
nlp = stanza.Pipeline('id', processors='tokenize')


import os
file_list=os.listdir()

from tqdm import tqdm


# 스파이시 영어 모델 로드
nlp = spacy.load('en_core_web_sm')

# 스탄자 인도네시아어 모델 로드
stanza_id = stanza.Pipeline('id', processors='tokenize')

2023-10-30 23:27:28 INFO: Downloading default packages for language: id (Indonesian) ...
2023-10-30 23:27:28 INFO: File exists: C:\Users\USER\stanza_resources\id\default.zip
2023-10-30 23:27:30 INFO: Finished downloading models and saved to C:\Users\USER\stanza_resources.
2023-10-30 23:27:30 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-30 23:27:30 WARNING: Language id package default expects mwt, which has been added
2023-10-30 23:27:30 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-10-30 23:27:30 INFO: Using device: cpu
2023-10-30 23:27:30 INFO: Loading: tokenize
2023-10-30 23:27:31 INFO: Loading: mwt
2023-10-30 23:27:31 INFO: Done loading processors!
2023-10-30 23:27:31 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-30 23:27:31 WARNING: Language id package default expects mwt, which has been added
2023-10-30 23:27:31 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-10-30 23:27:31 INFO: Using device: cpu
2023-10-30 23:27:31 INFO: Loading: tokenize
2023-10-30 23:27:31 INFO: Loading: mwt
2023-10-30 23:27:31 INFO: Done loading processors!


</br>
</br>
</br>

# 함수정의
- 불필요 텍스트 제거
- 메타데이터 명시
- 파일 읽기 (데이터 수집 날짜, 키워드 명시)

In [2]:

def cleansing(text):
    # "MMM DD, YYYY" 형식의 날짜와 ")"로 끝나는 부분을 찾아 제거하는 정규표현식
    pattern_1 = r'\w{3}\s\d{2},\s\d{4}\(.*?\)'
    pattern_2 = r'Loading Link to the original story\. Notes.*?sole discretion\.'
    pattern_3 = r'\b[A-Z][a-z]{2} \d{2}, \d{4} \(.*?by Newstex\)'
    
    # 정규표현식을 사용하여 패턴을 제거
    cleaned_text = re.sub(pattern_3, '', text, flags=re.DOTALL)
    
    # cleaned_text = re.sub(pattern_1, '', text)

    # 왼쪽 공백 제거
    cleaned_text = cleaned_text.lstrip()

    if text.endswith('sole discretion.'):
        cleaned_text= cleaned_text[:-1284]
    

    # 날짜 형식 제거 (예: Apr 08, 2013)
    cleaned_text = re.sub(r'\b[A-Z][a-z]{2} \d{2}, \d{4}\b', '', cleaned_text)
    # http:// 또는 https:// 형태 제거
    cleaned_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', cleaned_text)
    # 대괄호 안 숫자 제거
    cleaned_text = re.sub(r'\[\d+\]', '', cleaned_text)
    # <--> 형태 제거
    cleaned_text = re.sub(r'<.*?>', '', cleaned_text)
    

    # 정규표현식 패턴을 사용하여 해당 패턴 제거
    pic_pattern = re.compile(r'pic\.twitter\.com\/[A-Za-z0-9]+')
    cleaned_text = re.sub(pic_pattern, "", cleaned_text)


    # 한글 제거
    text_without_korean = re.sub('[가-힣]+', '', cleaned_text)
    ko_pattern = re.compile(r'[ㄱ-ㅎㅏ-ㅣ]+')
    text_without_korean = re.sub(ko_pattern, '', text_without_korean)
    
    
    # 링크 제거 (http:// 또는 https://로 시작하는 링크)
    text_without_links = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text_without_korean)
    if (text_without_links.startswith("(")) & (")" in text_without_links):
        end_index = text_without_links.index(')')
        text_without_links = text_without_links[end_index+1:]
    if text_without_links.startswith("(KoreanIndo: Delivered by Newstex)"):
        text_without_links = text_without_links[len("(KoreanIndo: Delivered by Newstex)"):]
    
    
    pattern = re.compile(r'(source|Source)')
    match = pattern.search(text_without_links)

    
    if match:
        text_without_links = re.split(r'(source|Source).*', text_without_links)[0].strip()
        
    else:
        pass
    text_without_links = text_without_links.replace("Loading", "")
    
    
    text_without_links = text_without_links.replace("[embedded content]", "")
    text_without_links = re.sub(r'\[ \d+\]:', '', text_without_links)
    return text_without_links


def cleansing_text(test):

    test = re.sub(r"Baca Juga:.*?\.", "", test) #Baca Juga 관련 문장 제거
    test = re.sub(r"Baca Juga :.*?\.", "", test)
    test = re.sub(r"Baca juga:.*?\.", "", test)
    test = re.sub(r"Baca juga.*?\.", "", test)
    test = re.sub(r"Baca juga :.*?\.", "", test)
    test = re.sub(r"baca juga:.*?\.", "", test)
    test = re.sub(r"baca juga :.*?\.", "", test)
    test = re.sub(r"baca Juga:.*?\.", "", test)
    test = re.sub(r"baca Juga :.*?\.", "", test)
    test = re.sub(r"Foto:.*?\.","",test) #Foto 관련 문장 제거
    test = re.sub(r"Written by:.*","",test) #Written by: 이후 모든 문장 제거
    test = re.sub(r"Written by :.*","",test)
    test = re.sub(r"written by:.*","",test)
    test = re.sub(r"written by :.*","",test)
    test = re.sub(r"written by .*","",test)
    test = re.sub(r"—- Written by :.*","",test)
    test = re.sub(r"—Written by :.*","",test)
    test = re.sub(r"\b[A-Z][a-z]{2} \d{2}, \d{4} \(.*?by Newstex\)","",test) #날짜 제거
    test = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+","",test) #링크 제거
    test = re.sub(r"\[\d+\]","",test) #대괄호 포함 내용 제거
    test = re.sub(r"\[ \d+\]","",test) #대괄호 포함 내용 제거
    test = re.sub(r":","",test)
    test = re.sub(r"<.*?>","",test)
    test = re.sub(r"//instagram.com/.*","",test) #//instagram.com/ 이후 모든 문장 제거
    test = re.sub(r"\b(?:\w+\W+){0,30}Soompi(?:\W+\w+){0,30}\b", "", test) #Soompi 기준 앞 뒤 단어 30개 제거
    test = re.sub(r"\b(?:\w+\W+){0,30}soompi(?:\W+\w+){0,30}\b","",test) #soompi
    test = re.sub(r'#\d+;', "", test) ##8216;, #8217; 등 제거
    test = re.sub(r'# \d+;', "", test)
    test = re.sub(r'#\d+ ;', "", test)
    test = re.sub(r'# \d+ ;', "", test)
        
    return test



def second_cleansing_text(dtest):

    dtest = re.sub(r"\[ \d+\]","",dtest)
    dtest = re.sub(r"Written by:.*","",dtest) #Written by: 이후 모든 문장 제거
    dtest = re.sub(r"Written by :.*","",dtest)
    dtest = re.sub(r"written by:.*","",dtest)
    dtest = re.sub(r"written by :.*","",dtest)
    dtest = re.sub(r"//instagram.com/.*","",dtest) #//instagram.com/ 이후 모든 문장 제거
    dtest = re.sub(r"\b(?:\w+\W+){0,30}Soompi(?:\W+\w+){0,30}\b", "", dtest) #Soompi 기준 앞 뒤 단어 30개 제거
    dtest = re.sub(r"\b(?:\w+\W+){0,30}soompi(?:\W+\w+){0,30}\b","",dtest) #soompi
    dtest = re.sub(r":","",dtest)
    dtest = re.sub(r"[一-龥]+","",dtest)

    return dtest

    

# 1 -> 000001, 10231 -> 010231 형태로 id number 바꾸기
def format_id(id_number):
    # Check if the value is NaN
    if pd.isna(id_number):
        return 'NaN'
    
    formatted_id = '{:06d}'.format(int(id_number))
    return formatted_id



#수집일 및 키워드 표기
def set_metadata(df):

    date_str = date

    # YYYY-MM-DD 형태로 수집 날짜 명시
    formatted_date = f"{date[:4]}-{date[4:6]}-{date[6:]}"
    print("collection date :", formatted_date)

    # 파일명에서 확장자(.csv)를 제외한 부분 선택
    prefix = file_name.split('.')[0]
    
    df['Doc_ID'] = prefix + "_" + df['ID'].apply(format_id).astype(str)
    df['Col_Date']= formatted_date
    df['Keyword']=keyword
    
    for idx in df.index :
        if type(df['Date'][idx]) == float:
            # Check if it's the first row
            if idx == df.index[0]:
                # Handle the first row differently. For example, set it to NaN or a default date.
                df['Date'][idx] = pd.NaT
            else:
                df['Date'][idx] = df['Date'][idx-1]



def read_file(file_name):
    global date, keyword
    
    # 파일명에서 날짜와 도시 이름을 추출하는 정규표현식 패턴
    pattern = re.compile(r'(\d{8})_newsdata_(.*)\.csv')
    match = re.match(pattern, file_name)

    if match:
        date = match.group(1)  # 추출한 날짜
        keyword = match.group(2)  # 추출한 도시 이름
        print("Date:", date)
        print("Keyword:", keyword)
    else:
        print("No match found")


    df= pd.read_csv(file_name)
    return df


# 'Article' Column에 있는 기사 본문의 metadata 명시 및 불필요 텍스트 제거 후 'Text' Column으로 생성
def preprocessing(df) :
    set_metadata(df)
    df['Text'] = df['Article'].apply(cleansing_text)
    df['Text'] = df['Text'].apply(cleansing)
    df['Text'] = df['Text'].apply(second_cleansing_text)
    
    

In [4]:
path = 'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\newsdata_all\\한국위주키워드'
os.chdir(path)
os.getcwd()

'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\newsdata_all\\한국위주키워드'

<br>
<br>

## 폴더의 파일들 읽어와서 텍스트 정제 후 데이터프레임 만들기

In [5]:
# 현재 폴더 내의 파일 리스트 얻기
file_list = os.listdir()

# 폴더 내 파일들에 대해 반복하여 CSV 파일 읽기
df_all= pd.DataFrame()
new_df = pd.DataFrame()

for file_name in tqdm(file_list):
    if file_name.endswith('.csv'):
        new_df = read_file(file_name)
        print(f'filename : {file_name}')
        print(f"length : {len(new_df)}")
        preprocessing(new_df)

    df_all =pd.concat([df_all, new_df], axis=0)
print("-----------------------------------\n")

# news_list에 있는 키워드가 들어간 요소들 추리기
news_list= ['KoreanIndo', 'Newstex Blogs', 'The Conversation (Indonesia Edition)', 'Mobile88.com', 'GlobeNewswire Indonesian',
             'ParsToday (Indonesian)', 'Antara News', 'Plus Company Updates(PCU)', 'Mongabay.com - Indonesian', 'Thomson Reuters ONE',
               'Koreana', 'Webnews - Indonesian','Tenders Monitor Africa-Asia', 'Berita Harian', 'ParsToday (Indonesian)', 'Berita Dalam Negeri',
                 'TendersInfo - Contract Awards', 'The Conversation (Indonesia Edition)', 'Antara News', 'Malaysia General News', 'Industry SnapShot',
                   'Plus Company Updates(PCU)', 'Berita Ekonomi', 'ASAPII Database', 'Channel NewsAsia', 'New Straits Times (Malaysia)', 'BruDirect',
                     'US Official News', 'Malaysia Economic News']
df_all= df_all[df_all['Newspaper'].isin(news_list)]

# 중복 제거 후 날짜 기준으로 정렬
df_all.drop_duplicates(subset='Text', inplace=True)
df_all.sort_values('Date', inplace=True)
df_all.reset_index(drop=True, inplace=True)

df_all

  0%|          | 0/58 [00:00<?, ?it/s]

Date: 20231005
Keyword: ADB
filename : 20231005_newsdata_ADB.csv
length : 147
collection date : 2023-10-05


  2%|▏         | 1/58 [00:02<02:28,  2.61s/it]C:\Users\USER\AppData\Local\Temp\ipykernel_9040\83953182.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'][idx] = df['Date'][idx-1]


Date: 20231005
Keyword: ASEAN
filename : 20231005_newsdata_ASEAN.csv
length : 951
collection date : 2023-10-05


  3%|▎         | 2/58 [00:42<22:41, 24.32s/it]

...

 98%|█████████▊| 57/58 [26:33<00:29, 29.94s/it]C:\Users\USER\AppData\Local\Temp\ipykernel_9040\83953182.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'][idx] = df['Date'][idx-1]


Date: 20231027
Keyword: Tapanuli ogong
filename : 20231027_newsdata_Tapanuli ogong.csv
length : 327
collection date : 2023-10-27


100%|██████████| 58/58 [26:39<00:00, 27.58s/it]


-----------------------------------



,ID,Source_File,Newspaper,Date,Length,Section,Author,Edition,Headline,Graphic,Article,Doc_ID,Col_Date,Keyword,Text
0,18,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,US Official News,2013-01-26,916 words,NaN,NaN,NaN,Washington: Frost & Sullivan Prediksi Industri...,False,The Frost & Sullivan has issued the following...,20231005_newsdata_ASEAN_000018,2023-10-05,ASEAN,The Frost & Sullivan has issued the following ...
1,30,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231011_dat...,New Straits Times (Malaysia),2013-03-12,380 words,Pg. 47; LOCAL,NaN,NaN,Miley tetap mahu kahwin,False,GAMBARNYA dirakam tanpa cincin pertunangan dan...,20231011_newsdata_Penyanyi StarBe_000030,2023-10-11,Penyanyi StarBe,GAMBARNYA dirakam tanpa cincin pertunangan dan...
2,17,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Malaysia Economic News,2013-03-28,182 words,NaN,NaN,NaN,MAYBANK SASAR CAWANGAN MUMBAI BEROPERASI MULAI...,False,Maybank jangka cawangan terbaharunya di Mumbai...,20231005_newsdata_ASEAN_000017,2023-10-05,ASEAN,Maybank jangka cawangan terbaharunya di Mumbai...
3,9,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231009_dat...,Malaysia General News,2013-04-07,663 words,NaN,NaN,NaN,"WATER DISRUPTION IN REMBAU, PD, TAMPIN, JEMPOL...",False,"Several areas in Rembau, Tampin, Kuala Pilah, ...",20231009_newsdata_Cina_000009,2023-10-09,Cina,"Several areas in Rembau, Tampin, Kuala Pilah, ..."
4,41,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231025_dat...,Malaysia General News,2013-04-16,686 words,NaN,NaN,NaN,LIST OF GE13 BN CANDIDATES FOR PERAK,False,Following is the list of Barisan Nasional (BN)...,20231025_newsdata_Pantai Mawun_000041,2023-10-25,Pantai Mawun,Following is the list of Barisan Nasional (BN)...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31086,451,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,ParsToday (Indonesian),2023-06-24,2610 words,NaN,Pars Today,NaN,"Dinamika Asia Tenggara, 24 Juni 2023",False,Dinamika di negara-negara Asia Tenggara pekan ...,20231027_newsdata_Banjarmasin_000451,2023-10-27,Banjarmasin,Dinamika di negara-negara Asia Tenggara pekan ...
31087,351,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Newstex Blogs,2023-06-26,1218 words,NaN,Sapariah Saturi,NaN,Kala Ratusan Kg Sisik Trenggiling Diamankan di...,False,"Jun 26, 2023( Mongabay.com - Indonesian: http:...",20231027_newsdata_Tapanuli ogong_000351,2023-10-27,Tapanuli ogong,Petugas Bareskrim Mabes Polri berhasil meng...
31088,2353,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Industry SnapShot,2023-06-27,17425 words,NaN,NaN,NaN,Indonesia Mining 27 Jun 23,False,#(1) This Week's News Emitennews.com - Harga E...,20231027_newsdata_Leebong Island_002353,2023-10-27,Leebong Island,#(1) This Week's News Emitennews.com - Harga E...
31089,2352,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Newstex Blogs,2023-06-27,2147 words,NaN,Jay Fajar,NaN,Kegiatan Konservasi Kini Bisa Dilakukan di Lau...,False,"Jun 27, 2023( Mongabay.com - Indonesian: http:...",20231027_newsdata_Leebong Island_002352,2023-10-27,Leebong Island,"Selama berabad-abad, laut lepas selalu menjadi..."


In [11]:
#기사 수집 실적
df_all["Length"] = df_all["Length"].str[:-5]

In [12]:
# NaN 값을 0으로 대체
df_all["Length"] = df_all["Length"].fillna(0)

In [13]:
df_all["Length"] = df_all["Length"].astype(int)

In [14]:
df_all[['Length']]

,Length
0,177
1,428
2,541
3,3546
4,511
...,...
15949,79
15950,89
15951,113
15952,183


In [15]:
df_all["Length"].sum()

5129488

<br>
<br>
<br>

# 문장 분리

In [6]:
def tokenizing(df):
    new_rows = []
    max_length = 5
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        article_text = row['Text']
        doc = nlp(article_text)
        sen_id = 1  # 문장 ID를 1로 초기화



        for sentence in doc.sents:  # spacy에서는 .sents로 문장에 접근
            sentence = re.sub(r'shared from .*', "", sentence.text)
            sentence = re.sub(r'Shared from .*', "", sentence)
            sentence = re.sub(r'shared by .*', "", sentence)
            sentence = re.sub(r'Shared by .*', "", sentence)
            sentence = re.sub(r'&#\d+;', "", sentence) #remove &#38;(문장 부호 유형)
            sentence = re.sub(r'#\d+;', "", sentence) #remove #8216;, #8217; etc(문장 부호 유형)
            sentence = re.sub(r'# \d+;', "", sentence)
            sentence = re.sub(r'#\d+ ;', "", sentence)
            sentence = re.sub(r'# \d+ ;', "", sentence)
            sentence = re.sub(r"'#\d+;[^']+'", "", sentence)
            sentence = re.sub(r"ig_mid=[A-Z0-9\-]+", "", sentence) # #38; 뒤에 오는 ig_mid= 형태의 문장 제거(문장 부호 유형)
            sentence = re.sub(r'\[ \d+ \] : .*',"",sentence) #remove [ 1 ] : [ 2 ] : [ 3 ] etc
            sentence = re.sub(r"Written by:.*","",sentence) #Written by: 이후 모든 문장 제거(출처&링크 유형)
            sentence = re.sub(r"Written by :.*","",sentence)
            sentence = re.sub(r"Written by.*","",sentence)
            sentence = re.sub(r"written by:.*","",sentence)
            sentence = re.sub(r"written by :.*","",sentence)
            sentence = re.sub(r"written by .*","",sentence)
            sentence = re.sub(r"—- Written by :.*","",sentence)
            sentence = re.sub(r"—Written by :.*","",sentence)
            sentence = re.sub(r"#main #main #main.*","",sentence) # #main 이후 모든 문장 제거
            sentence = re.sub(r"About these ads.*","",sentence) #About these ads. 이후 모든 문장 제거(광고 유형)
            sentence = re.sub(r"Allkpop.*","",sentence) #Allkpop 이후 모든 문장 제거(제거 시 가끔 이메일 주소도 같이 삭제)
            sentence = re.sub(r"allkpop.*","",sentence)
            sentence = re.sub(r"Indotrans.*","",sentence) #allkpop 바로 뒤에 나오는 indotrans 이후 모든 문장 제거
            sentence = re.sub(r"indotrans.*","",sentence)
            sentence = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.(com|net|kr)\b","",sentence) #remove email(.com, .net, .kr)
            sentence = re.sub(r"\([^@]+@[^\)]+\)","",sentence) #(@e-mail) 형태 문장 제거
            sentence = re.sub(r"\(@[^)]+\)","",sentence) #(@단어) 형태 문장 제거(SNS 계정 포함)
            sentence = re.sub(r"\( @[^)]+\)","",sentence) 
            sentence = re.sub(r"\(#[^)]+\)","",sentence) #(#단어) 형태 문장 제거
            sentence = re.sub(r"@\w+\b","",sentence) #@포함 바로 뒤에 오는 단어 제거
            sentence = re.sub(r"#\w+\b","",sentence) # #포함 바로 뒤에 오는 단어 제거
            sentence = re.sub(r"utm_medium.*","",sentence) #utm_medium 이후 모든 문장 제거 (Soompi 관련 text)
            sentence = re.sub(r"utm_campaign.*","",sentence)
            sentence = re.sub(r"Photo Credit.*","",sentence) #Photo Credit/credit 이후 모든 문장 재거
            sentence = re.sub(r"Photo credit.*","",sentence)
            sentence = re.sub(r"Article credit.*","",sentence)
            sentence = re.sub(r"Lihat videonya.*","",sentence) #광고문 뒷문장 모두 제거
            sentence = re.sub(r"lihat videonya.*","",sentence) 
            sentence = re.sub(r"^.*\d+\s*points\s*\d+.*$","",sentence) #'숫자 points 숫자' 형태의 문장 제거(고유명사 나열 유형)
            sentence = re.sub(r"^.*\d+\s*Points\s*\d+.*$","",sentence) 
            sentence = re.sub(r"^.*\d+\s*points\s*\.*$","",sentence)
            sentence = re.sub(r"^.*\d+\s*poin.*$","",sentence) # '숫자 poin' 형태의 문장 제거(고유명사 나열 유형)
            sentence = re.sub(r"^.*\d+\s*Poin.*$","",sentence)
            sentence = re.sub(r"^.*berikut\sini\.\s*$","",sentence) #텍스트 마지막에 존재하는 berikut ini. 가 포함된 문장 제거(인니어 광고문 유형)
            sentence = re.sub(r"^.*bawah ini.*$","",sentence) #di bawah ini가 포함된 문장 제거(인니어 광고문 유형)
            sentence = re.sub(r"^.*\.\.\.$","",sentence) # ...으로 끝나는 문장 제거(문장 잘림 유형)
            sentence = re.sub(r"\[.*?\]","",sentence) # 대괄호 및 단어 제거
            sentence = re.sub(r"www\.[a-zA-Z0-9-]+\.com","",sentence) # www.com 형태의 문장 제거
            sentence = re.sub(r"\b\w+\.com\b","",sentence) # .com 형태의 문장 제거
            sentence = re.sub(r"\([^)]*persen[^)]*\)","",sentence) #(persen...text) 형태의 문장 제거
            sentence = re.sub(r"\([^)]*Entertainment[^)]*\)","",sentence) #(Entertainment...text) 형태의 문장 제거
            sentence = re.sub(r".+\s*\.\.\.$","",sentence) #마지막이 ...로 끝나는 문장 모두 제거
            sentence = re.sub(r".+\s*….$","",sentence)
            sentence = re.sub(r"\(.*?\)","",sentence) #()포함 소괄호 안에 있는 텍스트까지 제거
            #하나씩 제거하는 정규표현식
            sentence = re.sub(r'gt;_lt;', '',sentence) #remove ;gt and ;lt
            sentence = re.sub(r'gt;',"",sentence)
            sentence = re.sub(r'lt;',"",sentence)
            sentence = re.sub(r'_lt;',"",sentence)
            sentence = re.sub(r"Advertisements","",sentence) #remove english
            sentence = re.sub(r"Instagrammable","",sentence)
            sentence = re.sub(r"instagrammable","",sentence)
            sentence = re.sub(r"LOL","",sentence)
            sentence = re.sub(r"LoL","",sentence)
            sentence = re.sub(r"gantiin","",sentence) #remove wrong spelling
            sentence = re.sub(r"content]","",sentence)
            sentence = re.sub(r"@ star1","",sentence) #@star1 이라는 잡지회사 제거
            sentence = re.sub(r"@star1","",sentence)
            sentence = re.sub(r"♥","",sentence) #특수문자 제거
            sentence = re.sub(r"▲","",sentence)
            sentence = re.sub(r"=","",sentence)
            sentence = re.sub(r"#","",sentence) 
            sentence = re.sub(r"@","",sentence) 
            sentence = re.sub(r"\[","",sentence) 
            sentence = re.sub(r"]","",sentence) 
            sentence = re.sub(r"~","",sentence) 
            sentence = re.sub(r"\^","",sentence) 
            sentence = re.sub(r"→","",sentence) 
            sentence = re.sub(r"-","",sentence)
            spacy_doc = nlp(sentence)  # Renamed to spacy_doc for clarity
            tokens = [token.text for sent in spacy_doc.sents for token in sent]
            
            # 문장, 토큰, 문장길이, Sen_ID 컬럼 추가
            new_row = row.copy()
            new_row['Sentence'] = sentence
            new_row['Token'] = tokens
            new_row['Sen_len'] = len(tokens)
            new_row['Sen_ID'] = f"{row['Doc_ID']}_sen{sen_id:06d}"
            
            new_rows.append(new_row)
            sen_id += 1  # 문장 ID 증가


    # 리스트 기호 없이 단어 단위로 토큰화
    new_df = pd.DataFrame(new_rows)
    new_df['Pub_Type']='Newspaper'
    new_df['Tokenized_Sentence']= new_df['Token'].apply(' '.join)

    # 'Sentence' 컬럼 중심으로 중복 제거
    new_df.drop_duplicates(subset='Sentence', inplace=True)
    new_df.reset_index(drop=True, inplace=True)

    # 공식 제출용 컬럼명으로 컬럼명 변환
    new_df.rename(columns={'Source_File' : 'Filename', 'Headline':'Title', 'Keyword':'Pub_Subj', 'Date': 'Pub_Date', 'Sen_len': 'Word_Count'}, inplace=True)
    
    new_df = new_df[['Doc_ID', 'Filename', 'Title', 'Pub_Type', 'Pub_Subj', 'Pub_Date', 'Col_Date', 'Sen_ID', 'Word_Count', 'Text', 'Sentence','Tokenized_Sentence' , 'Token']]
    new_df = new_df[new_df['Token'].apply(lambda x: len(x) > max_length)]
    return new_df


<br>
<br>

### 토큰화 완료된 데이터프레임

In [7]:
# 토큰화 완료된 최종 데이터프레임

df = tokenizing(df_all)
df

Processing rows: 100%|██████████| 31091/31091 [3:37:28<00:00,  2.38it/s]   


,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000001,29,The Frost & Sullivan has issued the following ...,The Frost & Sullivan has issued the following ...,The Frost & Sullivan has issued the following ...,"[The, Frost, &, Sullivan, has, issued, the, fo..."
1,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000002,43,The Frost & Sullivan has issued the following ...,"Vivek Vaidya, Vice President, Automotive & Tra...","Vivek Vaidya , Vice President , Automotive & T...","[Vivek, Vaidya, ,, Vice, President, ,, Automot..."
2,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000003,16,The Frost & Sullivan has issued the following ...,"Meskipun demikian, pertumbuhan tersebut diperk...","Meskipun demikian , pertumbuhan tersebut diper...","[Meskipun, demikian, ,, pertumbuhan, tersebut,..."
3,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000004,25,The Frost & Sullivan has issued the following ...,Permintaan terhadap segmen mobil penumpang di ...,Permintaan terhadap segmen mobil penumpang di ...,"[Permintaan, terhadap, segmen, mobil, penumpan..."
4,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000005,18,The Frost & Sullivan has issued the following ...,"Menurut Vaidya, pertumbuhan juga akan didorong...","Menurut Vaidya , pertumbuhan juga akan didoron...","[Menurut, Vaidya, ,, pertumbuhan, juga, akan, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
966041,20231027_newsdata_Leebong Island_002351,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Research shows how smugglers 'help' Indonesian...,Newspaper,Leebong Island,2023-06-29,2023-10-27,20231027_newsdata_Leebong Island_002351_sen000040,28,The United Nations defines people smuggling as...,"However, those in need for new papers have to ...","However , those in need for new papers have to...","[However, ,, those, in, need, for, new, papers..."
966042,20231027_newsdata_Leebong Island_002351,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Research shows how smugglers 'help' Indonesian...,Newspaper,Leebong Island,2023-06-29,2023-10-27,20231027_newsdata_Leebong Island_002351_sen000041,24,The United Nations defines people smuggling as...,"If pressed for time, migrants are unlikely to ...","If pressed for time , migrants are unlikely to...","[If, pressed, for, time, ,, migrants, are, unl..."
966043,20231027_newsdata_Leebong Island_002351,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Research shows how smugglers 'help' Indonesian...,Newspaper,Leebong Island,2023-06-29,2023-10-27,20231027_newsdata_Leebong Island_002351_sen000042,26,The United Nations defines people smuggling as...,Unless Indonesia and Malaysia address this pro...,Unless Indonesia and Malaysia address this pro...,"[Unless, Indonesia, and, Malaysia, address, th..."
966044,20231027_newsdata_Leebong Island_002351,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Research shows how smugglers 'help' Indonesian...,Newspaper,Leebong Island,2023-06-29,2023-10-27,20231027_newsdata_Leebong Island_002351_sen000043,10,The United Nations defines people smuggling as...,Wayne Palmer has received funding from Austral...,Wayne Palmer has received fun

In [8]:
import pandas as pd
import re

words_to_remove = ['/instagram', 'Written by', 'written by', 'WRITTEN BY', 'POWERED BY',
                    'ads', 'full credits', 'Full Credits', 'FULL CREDITS']

# Function to remove sentences containing specified words
def remove_sentences_with_words(text, words):
    # Split the text into sentences using a simple regex
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Initialize a list to store the filtered sentences
    filtered_sentences = []

    for sentence in sentences:
        # Check if any of the specified words exist in the sentence (case insensitive)
        if any(re.search(fr'\b{re.escape(word)}\b', sentence, re.IGNORECASE) for word in words):
            continue  # Skip this sentence if it contains any of the specified words
        else:
            filtered_sentences.append(sentence)

    # Join the filtered sentences back into a single text
    result = ' '.join(filtered_sentences)
    return result



In [9]:
# 데이터프레임에 적용
import numpy as np


df['Tokenized_Sentence'] = df['Tokenized_Sentence'].apply(remove_sentences_with_words, words=words_to_remove)

df['Tokenized_Sentence'].replace('', np.nan, inplace=True)
df = df.dropna()

df


,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000001,29,The Frost & Sullivan has issued the following ...,The Frost & Sullivan has issued the following ...,The Frost & Sullivan has issued the following ...,"[The, Frost, &, Sullivan, has, issued, the, fo..."
1,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000002,43,The Frost & Sullivan has issued the following ...,"Vivek Vaidya, Vice President, Automotive & Tra...","Vivek Vaidya , Vice President , Automotive & T...","[Vivek, Vaidya, ,, Vice, President, ,, Automot..."
2,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000003,16,The Frost & Sullivan has issued the following ...,"Meskipun demikian, pertumbuhan tersebut diperk...","Meskipun demikian , pertumbuhan tersebut diper...","[Meskipun, demikian, ,, pertumbuhan, tersebut,..."
3,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000004,25,The Frost & Sullivan has issued the following ...,Permintaan terhadap segmen mobil penumpang di ...,Permintaan terhadap segmen mobil penumpang di ...,"[Permintaan, terhadap, segmen, mobil, penumpan..."
4,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000005,18,The Frost & Sullivan has issued the following ...,"Menurut Vaidya, pertumbuhan juga akan didorong...","Menurut Vaidya , pertumbuhan juga akan didoron...","[Menurut, Vaidya, ,, pertumbuhan, juga, akan, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
966041,20231027_newsdata_Leebong Island_002351,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Research shows how smugglers 'help' Indonesian...,Newspaper,Leebong Island,2023-06-29,2023-10-27,20231027_newsdata_Leebong Island_002351_sen000040,28,The United Nations defines people smuggling as...,"However, those in need for new papers have to ...","However , those in need for new papers have to...","[However, ,, those, in, need, for, new, papers..."
966042,20231027_newsdata_Leebong Island_002351,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Research shows how smugglers 'help' Indonesian...,Newspaper,Leebong Island,2023-06-29,2023-10-27,20231027_newsdata_Leebong Island_002351_sen000041,24,The United Nations defines people smuggling as...,"If pressed for time, migrants are unlikely to ...","If pressed for time , migrants are unlikely to...","[If, pressed, for, time, ,, migrants, are, unl..."
966043,20231027_newsdata_Leebong Island_002351,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Research shows how smugglers 'help' Indonesian...,Newspaper,Leebong Island,2023-06-29,2023-10-27,20231027_newsdata_Leebong Island_002351_sen000042,26,The United Nations defines people smuggling as...,Unless Indonesia and Malaysia address this pro...,Unless Indonesia and Malaysia address this pro...,"[Unless, Indonesia, and, Malaysia, address, th..."
966044,20231027_newsdata_Leebong Island_002351,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231027_dat...,Research shows how smugglers 'help' Indonesian...,Newspaper,Leebong Island,2023-06-29,2023-10-27,20231027_newsdata_Leebong Island_002351_sen000043,10,The United Nations defines people smuggling as...,Wayne Palmer has received funding from Austral...,Wayne Palmer has received fun

In [10]:
# 문장 수 확인
df['Word_Count'].sum()

17459939

### 테스트웍스 전달 시 작업용 column 사용

In [13]:
# 토큰별 작업용 컬럼 생성
for i in range(df['Word_Count'].max()):
    new_column_name = f'Column_{i}'  # 새 열 이름 생성
    df[new_column_name]=np.nan

for idx in tqdm(df.index):
    for i, token in enumerate(df['Token'][idx]):
        new_column_name = f'Column_{i}'  # 새 열 이름 생성
        df[new_column_name][idx]=token

C:\Users\USER\AppData\Local\Temp\ipykernel_9040\2584547734.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name]=np.nan
C:\Users\USER\AppData\Local\Temp\ipykernel_9040\2584547734.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name]=np.nan
C:\Users\USER\AppData\Local\Temp\ipykernel_9040\2584547734.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [11]:
# 'Sen_ID'기준으로 정렬

df.sort_values('Sen_ID', inplace=True)
df.reset_index(drop=True, inplace=True)
df

C:\Users\USER\AppData\Local\Temp\ipykernel_6876\81318995.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values('Sen_ID', inplace=True)


<br>

### 합쳐진 원본데이터(보관용) pickle 형태로 만들기

In [11]:
#저장 경로 지정
path = 'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터 일일 구축량 데이터'
os.chdir(path)

In [12]:
# 날짜_newsdata_원천데이터 합본.pickle

df.to_pickle('한국위주키워드~20231027_news_assemble_unique.pickle')

In [18]:
#일일 구축량은 csv로 저장
df.to_csv('한국위주키워드~20231027_news_assemble_unique.csv', index = False, encoding = 'utf-8-sig')

<br>
<br>

## 날짜 단위로 기존 데이터 제거

In [8]:
## target_date 수정 후 사용

target_date='2023-08-23' 
df_new = df[df['Col_Date'] >= target_date]
df_new.sort_values('Sen_ID', inplace=True)
df_new.reset_index(drop=True, inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_19300\189799062.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0822.sort_values('Sen_ID', inplace=True)


<br>
<br>

# 가공된 데이터 제거

In [52]:
#20230904 조각 나누기 전 이미 가공된 파일 제외
df_finish = pd.read_excel('C:\\Users\\Juhee Kim\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\원천데이터\\가공완료데이터\\테스트웍스_인도네시아어 현황 관리 파일_230906.xlsx')
df_finish

,원천 전달 날짜,원천 파일명,가공 완료 날짜,주관기관 전달 파일,비고,파일명,Doc_ID,Pub_Type,Pub_Subj,Sen_ID,Word_Count
0,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,1,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000001,29
1,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,0,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000002,26
2,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,0,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000003,8
3,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,0,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000004,27
4,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,0,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000005,11
...,...,...,...,...,...,...,...,...,...,...,...
9582,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-09-04,NaN,NaN,0,20230808_newsdata_Korea_002210,Newspaper,Korea,20230808_newsdata_Korea_002210_000019,30
9583,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-09-04,NaN,NaN,0,20230808_newsdata_Korea_002210,Newspaper,Korea,20230808_newsdata_Korea_002210_000020,14
9584,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-09-04,NaN,NaN,0,20230808_newsdata_Korea_002210,Newspaper,Korea,20230808_newsdata_Korea_002210_000021,30
9585,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-09-04,NaN,NaN,0,20230808_newsdata_Korea_002210,Newspaper,Korea,20230808_newsdata_Korea_002210_000022,42


In [53]:
# 가공된 데이터에서 'Doc_ID' 리스트로 추출

df_filtered = df_finish['Doc_ID'].tolist()
df_filtered

['20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001001',
 '20230808_newsdata_Korea_001001',
 '20230808_newsdata_Korea_001001',
 '20230808_newsdata_Korea_001001',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001003',
 '20230808_newsdata_Korea_001003',
 '20230808_newsdata_Korea_001004',
 '20230808_newsdata_Korea_001004',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_

In [46]:
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,...,Column_237,Column_238,Column_239,Column_240,Column_241,Column_242,Column_243,Column_244,Column_245,Column_246
0,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000001,23,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000002,15,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000003,13,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000004,33,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000005,17,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392852,20230905_newsdata_Yeon_002695,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230905_dat...,Park Si Yeon Melahirkan Bayi Perempuan,Newspaper,Yeon,2013-09-24,2023-09-05,20230905_newsdata_Yeon_002695_sen000001,23,"Park Si Yeon telah melahirkan bayi perempuan, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392853,20230905_newsdata_Yeon_002695,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230905_dat...,Park Si Yeon Melahirkan Bayi Perempuan,Newspaper,Yeon,2013-09-24,2023-09-05,20230905_newsdata_Yeon_002695_sen000002,25,"Park Si Yeon telah melahirkan bayi perempuan, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392854,20230905_newsdata_Yeon_002695,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230905_dat...,Park Si Yeon Melahirkan Bayi Perempuan,Newspaper,Yeon,2013-09-24,2023-09-05,20230905_newsdata_Yeon_002695_sen000003,7,"Park Si Yeon telah melahirkan bayi perempuan, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392855,20230905_newsdata_Yeon_002695,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230905_dat...,Park Si Yeon Melahirkan Bayi Perempuan,Newspaper,Yeon,2013-09-24,2023-09-05,20230905_newsdata_Yeon_002695_sen000004,14,"Park Si Yeon telah melahirkan bayi perempuan, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# 'Doc_ID'기준으로 가공된 데이터 제외

df_final = df[~df['Doc_ID'].isin(df_filtered)]
df_final

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,...,Column_840,Column_841,Column_842,Column_843,Column_844,Column_845,Column_846,Column_847,Column_848,Column_849
0,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000001,38,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000002,13,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000003,33,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000004,17,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000005,18,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346847,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000004,7,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346848,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000005,15,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346849,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000006,6,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346850,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000007,10,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#20230922 Doc_ID 기준 중복 기사 제거
import pandas as pd
import os
path = 'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\원천데이터\\기사데이터\\20230911\\20230911-1'
os.chdir(path)
os.getcwd()

'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\원천데이터\\기사데이터\\20230911\\20230911-1'

In [20]:
df = pd.read_pickle('C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\newsdata_all\\20230926_news_assemble_unique.pickle')
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000001,22,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Kyuhyun Super Junior , Suho EXO , dan Lee Se...","[ , Kyuhyun, Super, Junior, ,, Suho, EXO, ,, d..."
1,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000002,15,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",Who Laughs' tayang perdana September lalu di K...,Who Laughs ' tayang perdana September lalu di ...,"[Who, Laughs, ', tayang, perdana, September, l..."
2,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000003,13,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Sebagai hasil dari keberhasilannya, tayangan u...","Sebagai hasil dari keberhasilannya , tayangan ...","[Sebagai, hasil, dari, keberhasilannya, ,, tay..."
3,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000004,33,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Berdasarkan novel karya Victor Hugo, 'The Man ...","Berdasarkan novel karya Victor Hugo , ' The Ma...","[Berdasarkan, novel, karya, Victor, Hugo, ,, '..."
4,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000005,17,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",Musikal ini mengkritik jatuhnya keadilan dan k...,Musikal ini mengkritik jatuhnya keadilan dan k...,"[Musikal, ini, mengkritik, jatuhnya, keadilan,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494292,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000072,18,Trashbusters tidak putus asa saat memandang tu...,Era yang memungkinkan apa saja dapat diantar m...,Era yang memungkinkan apa saja dapat diantar m...,"[Era, yang, memungkinkan, apa, saja, dapat, di..."
494293,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000073,23,Trashbusters tidak putus asa saat memandang tu...,Meski penggunaan kantong plastik sudah dilaran...,Meski penggunaan kantong plastik sudah dilaran...,"[Meski, penggunaan, kantong, plastik, sudah, d..."
494294,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000074,10,Trashbusters tidak putus asa saat memandang tu...,Hal-hal itu mungkin dapat memicu tawa sinis.,Hal - hal itu mungkin dapat memicu tawa sinis .,"[Hal, -, hal, itu, mungkin, dapat, memicu, taw..."
494295,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000075,9,Trashbusters tidak putus asa saat memandang tu...,"Meskipun demikian, mari kita mengucapkan sloga...","Meskipun demikian , mari kita mengucapkan slog...","[Meskipun, demikian, ,, mari, kita, mengucapka..."


In [21]:
df_1 = pd.read_csv('202300911_원천데이터(기사)_1.csv')
df_2 = pd.read_csv('202300911_원천데이터(기사)_2.csv')
df_3 = pd.read_csv('202300911_원천데이터(기사)_3.csv')
df_4 = pd.read_csv('202300911_원천데이터(기사)_4.csv')
df_5 = pd.read_csv('202300911_원천데이터(기사)_5.csv')
df_6 = pd.read_csv('202300911_원천데이터(기사)_6.csv')
df_7 = pd.read_csv('202300911_원천데이터(기사)_7.csv')


C:\Users\USER\AppData\Local\Temp\ipykernel_17356\4264928424.py:1: DtypeWarning: Columns (98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,3

In [22]:
df_combined = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7], ignore_index=True)
df_combined

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,...,Column_840,Column_841,Column_842,Column_843,Column_844,Column_845,Column_846,Column_847,Column_848,Column_849
0,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000001,38,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000002,13,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000003,33,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000004,17,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000005,18,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337688,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000004,7,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337689,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000005,15,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337690,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000006,6,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337691,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000007,10,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
unique_doc_ids = df[~df['Doc_ID'].isin(df_combined['Doc_ID'])][['Doc_ID']].reset_index(drop=True)
unique_doc_ids

,Doc_ID
0,20230808_newsdata_Korea_000400
1,20230808_newsdata_Korea_000400
2,20230808_newsdata_Korea_000400
3,20230808_newsdata_Korea_000400
4,20230808_newsdata_Korea_000400
...,...
103423,20230920_newsdata_Koreana2_000270
103424,20230920_newsdata_Koreana2_000270
103425,20230920_newsdata_Koreana2_000270
103426,20230920_newsdata_Koreana2_000270


In [24]:
unique_df = df[~df['Doc_ID'].isin(df_combined['Doc_ID'])].reset_index(drop=True)
unique_df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000001,27,"Saat ini, banyak kafe-kafe baru yang berlomba ...","Saat ini, banyak kafe-kafe baru yang berlomba ...","Saat ini , banyak kafe - kafe baru yang berlom...","[Saat, ini, ,, banyak, kafe, -, kafe, baru, ya..."
1,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000002,28,"Saat ini, banyak kafe-kafe baru yang berlomba ...",Dengan hanya sebulan setelah dibuka di daerah ...,Dengan hanya sebulan setelah dibuka di daerah ...,"[Dengan, hanya, sebulan, setelah, dibuka, di, ..."
2,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000003,31,"Saat ini, banyak kafe-kafe baru yang berlomba ...",Para penggemar drama Korea pasti akan teringat...,Para penggemar drama Korea pasti akan teringat...,"[Para, penggemar, drama, Korea, pasti, akan, t..."
3,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000004,20,"Saat ini, banyak kafe-kafe baru yang berlomba ...","Karena ini tempat baru, maka ada antrian panja...","Karena ini tempat baru , maka ada antrian panj...","[Karena, ini, tempat, baru, ,, maka, ada, antr..."
4,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000005,46,"Saat ini, banyak kafe-kafe baru yang berlomba ...",Dengan pemandangan menakjubkan dari lampu yang...,Dengan pemandangan menakjubkan dari lampu yang...,"[Dengan, pemandangan, menakjubkan, dari, lampu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103423,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000072,18,Trashbusters tidak putus asa saat memandang tu...,Era yang memungkinkan apa saja dapat diantar m...,Era yang memungkinkan apa saja dapat diantar m...,"[Era, yang, memungkinkan, apa, saja, dapat, di..."
103424,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000073,23,Trashbusters tidak putus asa saat memandang tu...,Meski penggunaan kantong plastik sudah dilaran...,Meski penggunaan kantong plastik sudah dilaran...,"[Meski, penggunaan, kantong, plastik, sudah, d..."
103425,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000074,10,Trashbusters tidak putus asa saat memandang tu...,Hal-hal itu mungkin dapat memicu tawa sinis.,Hal - hal itu mungkin dapat memicu tawa sinis .,"[Hal, -, hal, itu, mungkin, dapat, memicu, taw..."
103426,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000075,9,Trashbusters tidak putus asa saat memandang tu...,"Meskipun demikian, mari kita mengucapkan sloga...","Meskipun demikian , mari kita mengucapkan slog...","[Meskipun, demikian, ,, mari, kita, mengucapka..."


In [25]:
unique_df = unique_df.to_csv('20230926_원천데이터(기사).csv', index = False, encoding = 'utf-8-sig')

In [35]:
df = pd.read_csv('20230926_원천데이터(기사).csv')
df.tail()

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
103423,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000072,18,Trashbusters tidak putus asa saat memandang tu...,Era yang memungkinkan apa saja dapat diantar m...,Era yang memungkinkan apa saja dapat diantar m...,"['Era', 'yang', 'memungkinkan', 'apa', 'saja',..."
103424,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000073,23,Trashbusters tidak putus asa saat memandang tu...,Meski penggunaan kantong plastik sudah dilaran...,Meski penggunaan kantong plastik sudah dilaran...,"['Meski', 'penggunaan', 'kantong', 'plastik', ..."
103425,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000074,10,Trashbusters tidak putus asa saat memandang tu...,Hal-hal itu mungkin dapat memicu tawa sinis.,Hal - hal itu mungkin dapat memicu tawa sinis .,"['Hal', '-', 'hal', 'itu', 'mungkin', 'dapat',..."
103426,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000075,9,Trashbusters tidak putus asa saat memandang tu...,"Meskipun demikian, mari kita mengucapkan sloga...","Meskipun demikian , mari kita mengucapkan slog...","['Meskipun', 'demikian', ',', 'mari', 'kita', ..."
103427,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000076,33,Trashbusters tidak putus asa saat memandang tu...,"Kemudian, mari kita mengingatkan diri kita bah...","Kemudian , mari kita mengingatkan diri kita ba...","['Kemudian', ',', 'mari', 'kita', 'mengingatka..."


In [80]:
#20231005-20231010 원시데이터 원천데이터 50000개로 나누기
df = pd.read_pickle('C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\newsdata_all\\20231005~20231010\\20231005-20231010_news_assemble_unique.pickle')
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000001,29,The Frost & Sullivan has issued the following ...,The Frost & Sullivan has issued the following ...,The Frost & Sullivan has issued the following ...,"[The, Frost, &, Sullivan, has, issued, the, fo..."
1,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000002,43,The Frost & Sullivan has issued the following ...,"Vivek Vaidya, Vice President, Automotive & Tra...","Vivek Vaidya , Vice President , Automotive & T...","[Vivek, Vaidya, ,, Vice, President, ,, Automot..."
2,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000003,16,The Frost & Sullivan has issued the following ...,"Meskipun demikian, pertumbuhan tersebut diperk...","Meskipun demikian , pertumbuhan tersebut diper...","[Meskipun, demikian, ,, pertumbuhan, tersebut,..."
3,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000004,29,The Frost & Sullivan has issued the following ...,Permintaan terhadap segmen mobil penumpang di ...,Permintaan terhadap segmen mobil penumpang di ...,"[Permintaan, terhadap, segmen, mobil, penumpan..."
4,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000005,20,The Frost & Sullivan has issued the following ...,"Menurut Vaidya, pertumbuhan juga akan didorong...","Menurut Vaidya , pertumbuhan juga akan didoron...","[Menurut, Vaidya, ,, pertumbuhan, juga, akan, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
628261,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000011,24,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,"""Dan untuk hari terakhir, tanggal 14 Mei 2023 ...",""" Dan untuk hari terakhir , tanggal 14 Mei 202...","["", Dan, untuk, hari, terakhir, ,, tanggal, 14..."
628262,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000012,19,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,"Sebagai informasi, Pemilu 2024 diikuti 18 parp...","Sebagai informasi , Pemilu 2024 diikuti 18 par...","[Sebagai, informasi, ,, Pemilu, 2024, diikuti,..."
628263,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000013,16,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,Sebanyak 17 parpol ditetapkan pada 14 Desember...,Sebanyak 17 parpol ditetapkan pada 14 Desember...,"[Sebanyak, 17, parpol, ditetapkan, pada, 14, D..."
628264,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000014,25,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,Sedangkan satu parpol lain ditetapkan menyusul...,Sedangkan satu parpol lain ditetapkan menyusul...,"[Sedangkan, satu, parpol, lain, ditetapkan, me..."


In [81]:
df.reset_index(drop=True, inplace=True)
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000001,29,The Frost & Sullivan has issued the following ...,The Frost & Sullivan has issued the following ...,The Frost & Sullivan has issued the following ...,"[The, Frost, &, Sullivan, has, issued, the, fo..."
1,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000002,43,The Frost & Sullivan has issued the following ...,"Vivek Vaidya, Vice President, Automotive & Tra...","Vivek Vaidya , Vice President , Automotive & T...","[Vivek, Vaidya, ,, Vice, President, ,, Automot..."
2,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000003,16,The Frost & Sullivan has issued the following ...,"Meskipun demikian, pertumbuhan tersebut diperk...","Meskipun demikian , pertumbuhan tersebut diper...","[Meskipun, demikian, ,, pertumbuhan, tersebut,..."
3,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000004,29,The Frost & Sullivan has issued the following ...,Permintaan terhadap segmen mobil penumpang di ...,Permintaan terhadap segmen mobil penumpang di ...,"[Permintaan, terhadap, segmen, mobil, penumpan..."
4,20231005_newsdata_ASEAN_000018,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,Washington: Frost & Sullivan Prediksi Industri...,Newspaper,ASEAN,2013-01-26,2023-10-05,20231005_newsdata_ASEAN_000018_sen000005,20,The Frost & Sullivan has issued the following ...,"Menurut Vaidya, pertumbuhan juga akan didorong...","Menurut Vaidya , pertumbuhan juga akan didoron...","[Menurut, Vaidya, ,, pertumbuhan, juga, akan, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
597560,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000011,24,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,"""Dan untuk hari terakhir, tanggal 14 Mei 2023 ...",""" Dan untuk hari terakhir , tanggal 14 Mei 202...","["", Dan, untuk, hari, terakhir, ,, tanggal, 14..."
597561,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000012,19,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,"Sebagai informasi, Pemilu 2024 diikuti 18 parp...","Sebagai informasi , Pemilu 2024 diikuti 18 par...","[Sebagai, informasi, ,, Pemilu, 2024, diikuti,..."
597562,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000013,16,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,Sebanyak 17 parpol ditetapkan pada 14 Desember...,Sebanyak 17 parpol ditetapkan pada 14 Desember...,"[Sebanyak, 17, parpol, ditetapkan, pada, 14, D..."
597563,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000014,25,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,Sedangkan satu parpol lain ditetapkan menyusul...,Sedangkan satu parpol lain ditetapkan menyusul...,"[Sedangkan, satu, parpol, lain, ditetapkan, me..."


In [82]:
df = df.iloc[550001:]
df


,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
550001,20231005_newsdata_Nusantara_001178,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,"Pindang, Jejak Akulturasi Budaya Masyarakat Su...",Newspaper,Nusantara,2022-11-19,2023-10-05,20231005_newsdata_Nusantara_001178_sen000020,25,Bumbu-bumbu yang digunakan dalam masakan pi...,"Ikan air tawar yang digunakan umumnya baung, t...","Ikan air tawar yang digunakan umumnya baung , ...","[Ikan, air, tawar, yang, digunakan, umumnya, b..."
550002,20231005_newsdata_Nusantara_001178,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,"Pindang, Jejak Akulturasi Budaya Masyarakat Su...",Newspaper,Nusantara,2022-11-19,2023-10-05,20231005_newsdata_Nusantara_001178_sen000021,28,Bumbu-bumbu yang digunakan dalam masakan pi...,"Baca Sungai, Pindang Ikan, dan Akulturasi Per...","Baca Sungai , Pindang Ikan , dan Akulturasi ...","[Baca, Sungai, ,, Pindang, Ikan, ,, dan, Akult..."
550003,20231005_newsdata_Nusantara_001178,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,"Pindang, Jejak Akulturasi Budaya Masyarakat Su...",Newspaper,Nusantara,2022-11-19,2023-10-05,20231005_newsdata_Nusantara_001178_sen000022,20,Bumbu-bumbu yang digunakan dalam masakan pi...,"Secara umum bumbu pindang terdiri cabai, bawan...","Secara umum bumbu pindang terdiri cabai , bawa...","[Secara, umum, bumbu, pindang, terdiri, cabai,..."
550004,20231005_newsdata_Nusantara_001178,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,"Pindang, Jejak Akulturasi Budaya Masyarakat Su...",Newspaper,Nusantara,2022-11-19,2023-10-05,20231005_newsdata_Nusantara_001178_sen000023,15,Bumbu-bumbu yang digunakan dalam masakan pi...,Semua bumbu utama ini merupakan jenis yang umu...,Semua bumbu utama ini merupakan jenis yang umu...,"[Semua, bumbu, utama, ini, merupakan, jenis, y..."
550005,20231005_newsdata_Nusantara_001178,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231005_dat...,"Pindang, Jejak Akulturasi Budaya Masyarakat Su...",Newspaper,Nusantara,2022-11-19,2023-10-05,20231005_newsdata_Nusantara_001178_sen000024,21,Bumbu-bumbu yang digunakan dalam masakan pi...,"Jika merujuk asal, tanaman tersebut bersumber ...","Jika merujuk asal , tanaman tersebut bersumber...","[Jika, merujuk, asal, ,, tanaman, tersebut, be..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
597560,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000011,24,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,"""Dan untuk hari terakhir, tanggal 14 Mei 2023 ...",""" Dan untuk hari terakhir , tanggal 14 Mei 202...","["", Dan, untuk, hari, terakhir, ,, tanggal, 14..."
597561,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000012,19,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,"Sebagai informasi, Pemilu 2024 diikuti 18 parp...","Sebagai informasi , Pemilu 2024 diikuti 18 par...","[Sebagai, informasi, ,, Pemilu, 2024, diikuti,..."
597562,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000013,16,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,Sebanyak 17 parpol ditetapkan pada 14 Desember...,Sebanyak 17 parpol ditetapkan pada 14 Desember...,"[Sebanyak, 17, parpol, ditetapkan, pada, 14, D..."
597563,20231006_newsdata_Jakarta_017319,C:/Users/USER/Desktop/기사 데이터_로컬환경/20231006_dat...,KPU RI Terima Pendaftaran Bacaleg Pemilu 2024,Newspaper,Jakarta,2023-05-01,2023-10-06,20231006_newsdata_Jakarta_017319_sen000014,25,Komisi Pemilihan Umum (KPU) RI mengumumkan pen...,Sedangkan satu parpol lain ditetapkan menyusul...,Sedangkan satu parpol lain ditetapkan menyusu

In [79]:
df = df.to_csv('20231011_원천데이터(기사)_22.csv',index = False, encoding = 'utf-8-sig')

<br>
<br>

# 완료된 데이터프레임 pickle로 변환

In [55]:

df_final.to_pickle('20230911_원천데이터(기사).pickle')

In [12]:
df_final=pd.read_pickle('20230906_원천데이터(기사).pickle')

In [13]:
df_final.sort_values('Sen_ID', inplace=True)
df_final.reset_index(drop=True, inplace=True)

<br>
<br>

## 조각 나누기

In [56]:
chunk_size = 50000

# 데이터프레임을 작은 조각으로 나눈다
chunks = [df_final[i:i+chunk_size] for i in range(0, len(df_final), chunk_size)]

# 작은 조각들을 순회하면서 CSV 파일로 저장
for idx, chunk in enumerate(chunks):
    filename = f"202300911_원천데이터(기사)_{idx+1}.csv"
    chunk.to_csv('../'+filename,encoding='utf-8-sig' ,index=False)

print("CSV 파일로 저장이 완료되었습니다.")

CSV 파일로 저장이 완료되었습니다.


In [51]:
'''
원시데이터/원천데이터/날짜_원천데이터(기사)/ 폴더 안에 최종 csv파일 만들기

import os
import datetime

now = datetime.datetime.now()
now_format = now.strftime("%Y%m%d")

os.mkdir(f"../원천데이터/{now_format}_원천데이터(기사)")

chunk_size = 50000

# 데이터프레임을 작은 조각으로 나눈다
chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

# 작은 조각들을 순회하면서 CSV 파일로 저장
for idx, chunk in enumerate(chunks):
    filename = f"20230821_원천데이터(기사)_{idx+1}.csv"
    chunk.to_csv(f'../원천데이터/{now_format}_원천데이터(기사)/'+filename,encoding='utf-8-sig' ,index=False)

print("CSV 파일로 저장이 완료되었습니다.")
'''

<br>
<br>
<br>

### Pickle 파일 정리
- 2023mmdd_news_assemble_unique.pickle : 토큰화 완료된 파일
- 20230904_원천데이터(기사).pickle : 위의 파일에서 가공된 데이터 제외한 파일
> unique 붙은것은 토큰화 완료된 것


### csv 파일 정리
- 상위폴더의 2023mmdd_원천데이터(기사)_{idx+1}.csv : 최종본